Решаем задачу бинарной классификации . Для оценки качества модели выберем метрику ROC-AUC, так как она:

Устойчива к дисбалансу классов.
Показывает, насколько хорошо модель различает два класса (выжившие и невыжившие).


In [10]:
# Импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.dummy import DummyClassifier

# Загружаем данные из CSV-файла
df = pd.read_csv('train.csv')

# Выводим первые строки для проверки
display(df.head())

# Разделяем признаки и целевую переменную
X = df.drop(columns=['Survived'])
y = df['Survived']

# Разделяем данные на тренировочную и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Проверяем размеры выборок
print(f"Размер тренировочной выборки: {X_train.shape}\n")
print(f"Размер тестовой выборки: {X_test.shape}\n\n")

# Создаем DummyClassifier (бейзлайн)
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=42)
dummy_clf.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_dummy_pred = dummy_clf.predict(X_test)

# Выводим отчет с учетом zero_division
print("Качество бейзлайна:")
print(classification_report(y_test, y_dummy_pred, zero_division=0))

# Вычисляем ROC-AUC
y_dummy_proba = dummy_clf.predict_proba(X_test)[:, 1]
print(f"ROC-AUC бейзлайна: {roc_auc_score(y_test, y_dummy_proba):.4f}\n\n")


# Определяем числовые и категориальные признаки
numeric_features = ['Age', 'Fare']
categorical_features = ['Sex', 'Embarked', 'Pclass']

# Создаем ColumnTransformer для предобработки данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),  # Заполняем пропуски средним значением
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Преобразуем категориальные признаки
    ]
)

# Создаем пайплайн: предобработка + модель
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Предобработка данных
    ('scaler', StandardScaler()),    # Масштабирование признаков
    ('classifier', LogisticRegression(random_state=42))  # Логистическая регрессия
])

# Обучаем модель через пайплайн
pipeline.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

# Оцениваем качество модели
print("Классификационный отчет:")
print(classification_report(y_test, y_pred))

# Вычисляем ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC модели: {roc_auc:.4f}")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Размер тренировочной выборки: (712, 11)

Размер тестовой выборки: (179, 11)


Качество бейзлайна:
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       110
           1       0.00      0.00      0.00        69

    accuracy                           0.61       179
   macro avg       0.31      0.50      0.38       179
weighted avg       0.38      0.61      0.47       179

ROC-AUC бейзлайна: 0.5000


Классификационный отчет:
              precision    recall  f1-score   support

           0       0.80      0.86      0.83       110
           1       0.75      0.65      0.70        69

    accuracy                           0.78       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.78      0.78      0.78       179

ROC-AUC модели: 0.8340


Анализ результатов
Бейзлайн:
ROC-AUC ≈ 0.5 (случайные предсказания).
Precision, recall и f1-score показывают низкое качество.
ML-модель:
ROC-AUC > 0.8 (хорошее качество).
Precision, recall и f1-score значительно лучше, чем у бейзлайна.